## BUSINESS UNDERSTANDING

## 1.1 Introduction

The global marketplace has witnessed everchanging trends in different industries, and the real estate sector is no exception.To remain relevant and increase profitability in the Real estate market it is advantageous for leading Real Estate companies to study this trends so as to stay ahead. Happy Homes Partners are investors in the real estate market and have acknowledged that data science plays a pivotal role by uncovering hidden investment opportunities and steering clients towards profitable ventures.

## 1.2 Business Problem

Happy Homes Partners is looking to expand their property investments and are looking for optimal locations for their investment. We as Tech to Tech hub have a primary objective to facilitate informed real estate investment decisions for the Happy Homes Partners. Using the extensive housing dataset from Zillow, covering the period from April 1996 to April 2018, the project seeks to pinpoint the top 5 zip codes with the most promising investment potential.

## 1.3 Objectives

1. To identify the top 5 best zipcode for Happy Homes Partners to consider?

2. To identify the top 5 zip codes with the highest ROI.

3. To develop time series models to forecast real estate prices for different zip codes over various time horizons.

4. To establish cities that are optimal for both short-term and long-term investment

## DATA UNDERSTANDING

We will be using the Zillow Housing dataset, a CSV file for this project.
The dataset covers a large period from April 1996 to April 2018, allowing us to study long-term trends and different market conditions. It's a valuable resource for analyzing how prices have changed over time and predicting future trends.

## 2.1 Summary of the Dataset

RegionID - This is unique Id for the Regions.

SizeRank - This is the ranking done based on the size of the Region.

RegionName - This field contains the zip code of the Region.

RegionType - Type of Region is Zip.

StateName - State.

City - This column provide the specific City Name of Housing Data.

Metro - This provide the name of the metro city around that Region.

County Name - This is the County Name for that Region.

Months Column - These Columns contains the prices of Region for every month.

## 2.2 Data Inspection

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.filterwarnings('ignore')

# Custom Options for displaying rows.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',100)

In [2]:
# load the datasets
df = pd.read_csv('zillow_data.csv')

# View the first 5 rows
df.head()


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,...,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,344400.0,345700.0,346700.0,347800.0,349000.0,350400.0,352000.0,353900.0,356200.0,358800.0,361800.0,365700.0,370200.0,374700.0,378900.0,383500.0,388300.0,393300.0,398500.0,403800.0,409100.0,414600.0,420100.0,426200.0,432600.0,438600.0,444200.0,450000.0,455900.0,462100.0,468500.0,475300.0,482500.0,490200.0,...,863900.0,872900.0,883300.0,889500.0,892800,893600,891300,889900,891500,893000,893000,895000,901200,909400,915000,916700,917700,919800,925800,937100,948200,951000,952500,958600,966200,970400,973900,974700,972600,974300,980800,988000,994700,998700,997000,993700,991300,989200,991300,999100,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,235400.0,233300.0,230600.0,227300.0,223400.0,219600.0,215800.0,211100.0,205700.0,200900.0,196800.0,193600.0,191400.0,190400.0,190800.0,192700.0,196000.0,201300.0,207400.0,212200.0,214600.0,215100.0,213400.0,210200.0,206100.0,202100.0,198800.0,196100.0,194100.0,193400.0,193400.0,193100.0,192700.0,193000.0,193700.0,194800.0,196100.0,197800.0,199700.0,201900.0,...,234200.0,235400.0,236600.0,238500.0,240500,242600,244700,246300,247600,249600,251400,253000,255200,258000,261200,264700,268400,271400,273600,275200,276400,277000,277900,280000,282600,285400,288400,290800,292000,292800,293700,295200,297000,299000,300800,301800,302800,304400,306200,307000,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,210700.0,208300.0,205500.0,202500.0,199800.0,198300.0,197300.0,195400.0,193000.0,191800.0,191800.0,193000.0,195200.0,198400.0,202800.0,208000.0,213800.0,220700.0,227500.0,231800.0,233400.0,233900.0,233500.0,233300.0,234300.0,237400.0,242800.0,250200.0,258600.0,268000.0,277000.0,283600.0,288500.0,293900.0,299200.0,304300.0,308600.0,311400.0,312300.0,311900.0,...,282100.0,284200.0,286000.0,288300.0,290700,293300,295900,298300,300200,301300,301700,302400,303600,306200,309100,311900,314100,316300,319000,322000,324300,326100,327300,327000,327200,328500,329800,330000,329000,327800,326700,325500,324700,324500,323700,322300,320700,320000,320000,320900,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,504600.0,505500.0,505700.0,505300.0,504200.0,503600.0,503400.0,502200.0,500000.0,497900.0,496300.0,495200.0,494700.0,494900.0,496200.0,498600.0,502000.0,507600.0,514900.0,522200.0,529500.0,537900.0,546900.0,556400.0,566100.0,575600.0,584800.0,593500.0,601600.0,610100.0,618600.0,625600.0,631100.0,636600.0,642100.0,647600.0,653300.0,659300.0,665800.0,672900.0,...,1149900.0,1155200.0,1160100.0,1163300.0,1167700,1173900,1175100,1173500,1175500,1178500,1176400,1174600,1178500,1185700,1192900,1198800,1200400,1198900,1200200,1207400,1218600,1226600,1230700,1235400,1241300,1245700,1247000,1246700,1245700,1246000,1247700,1252900,1260900,1267900,1272600,1276600,1280300,1282500,1286000,1289000,1289800,1287700,1287400,1291500,1296600,129900

In [3]:
# View last 5 rows
df.tail()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,...,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,93700.0,93400.0,93200.0,93000.0,92900.0,92700.0,92600.0,92600.0,92600.0,92800.0,93100.0,93500.0,94000.0,94500.0,95100.0,95800.0,96500.0,97100.0,97800.0,98500.0,99200.0,100000.0,100800.0,101700.0,102600.0,103400.0,104200.0,104900.0,105600.0,106300.0,106800.0,107300.0,107800.0,108400.0,109000.0,109600.0,110200.0,110800.0,111400.0,112000.0,...,187600.0,187300.0,187300.0,188100.0,189600,190800,189800,188500,189400,191100,192400,193300,193400,192000,191200,190900,190900,191100,191900,191400,190700,190300,189500,188700,188600,189400,189600,189000,189000,190200,190700,190600,190300,193200,200200,206900,209600,211500,213600,215400,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,92200.0,92100.0,91900.0,91700.0,91300.0,90900.0,90500.0,90100.0,89800.0,89600.0,89400.0,89300.0,89200.0,89100.0,88900.0,88700.0,88400.0,88100.0,87700.0,87400.0,87200.0,87100.0,87100.0,87100.0,87200.0,87300.0,87300.0,87400.0,87300.0,87300.0,87200.0,87300.0,87400.0,87600.0,87900.0,88300.0,88700.0,89200.0,89600.0,90000.0,...,180000.0,180000.0,181400.0,181900.0,182200,183300,185800,187700,187500,184900,183000,181900,182300,184400,186300,188300,190800,191800,189500,187700,186800,186700,186900,186700,186400,186500,186700,186000,185400,184100,184500,186100,188700,190200,190900,191000,192200,194300,195500,197300,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,57700.0,58000.0,58200.0,58400.0,58700.0,59100.0,59500.0,59900.0,60300.0,60800.0,61300.0,62000.0,62600.0,63200.0,63800.0,64500.0,65200.0,65900.0,66700.0,67400.0,68200.0,69000.0,69800.0,70700.0,71500.0,72300.0,73000.0,73700.0,74300.0,74900.0,75500.0,76000.0,76400.0,76900.0,77300.0,77700.0,78000.0,78200.0,78400.0,78400.0,...,105100.0,105200.0,104800.0,104200.0,104200,104700,104600,104600,104800,104800,104600,104400,104400,105000,105900,106800,107500,107700,107800,108300,108600,108700,108900,109800,110700,111400,111900,113500,115200,116500,117600,118800,119600,120000,120500,120900,121200,120800,120700,121100,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,195000.0,196300.0,197700.0,199100.0,200700.0,202600.0,204900.0,207100.0,209100.0,211200.0,213400.0,215800.0,218300.0,221000.0,223900.0,226900.0,230300.0,234300.0,238700.0,243000.0,247300.0,251800.0,256300.0,260900.0,265500.0,270100.0,274500.0,278900.0,283200.0,287900.0,292700.0,297100.0,301100.0,305000.0,308700.0,312400.0,315900.0,319200.0,322400.0,325500.0,...,528900.0,532000.0,535200.0,538500.0,538300,543700,551400,552800,547100,537900,520700,517500,538300,552700,545800,536100,538300,537800,539000,543500,556000,559000,556400,549000,547700,549600,553100,555800,562700,567000,566700,569100,575800,581700,585400,590700,588000,592500,615900,647300,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400
14722,95851,89155,Mesquite,NV,Las Vegas,Clark,14723,176400.0,176300.0,176100.0,17600

In [4]:
# Look at the shape
df.shape

(14723, 272)

Our data has 14723 rows and 272 columns. The data is in a wide Format, evident by having columns 8 onwards being with dates. 

In [5]:
# Looking at more information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


Our data has 14723 entries ranging from 0 to 14722 with 272 columns with 3 main data types
1. 219 columns with floating point numbers
2. 49 columns with interger data types
3. 4 columns with object data types

In [6]:
# Looking at descriptive statistics of our dataset
df.describe()

,RegionID,RegionName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,...,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
count,14723.000000,14723.000000,14723.000000,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368500e+04,1.368500e+04,1.368500e+04,1.368500e+04,1.368500e+04,1.368500e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,...,1.466700e+04,1.466700e+04,1.466700e+04,1.466700e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04
mean,81075.010052,48222.348706,7362.000000,1.182991e+05,1.184190e+05,1.185374e+05,1.186531e+05,1.187803e+05,1.189275e+05,1.191205e+05,1.193454e+05,1.196851e+05,1.201213e+05,1.205442e+05,1.208966e+05,1.212682e+05,1.216559e+05,1.220531e+05,1.224563e+05,1.228799e+05,1.233274e+05,1.238231e+05,1.243537e+05,1.250026e+05,1.257367e+05,1.264508e+05,1.270863e+05,1.277364e+05,1.283850e+05,1.290327e+05,1.296764e+05,1.303245e+05,1.309826e+05,1.316781e+05,1.323993e+05,1.332247e+05,1.341336e+05,1.350209e+05,1.358416e+05,1.366838e+05,1.375321e+05,1.383918e+05,1.392652e+05,1.401606e+05,1.410848e+05,1.420701e+05,1.431040e+05,1.442613e+05,1.455221e+05,1.467723e+05,...,2.250826e+05,2.261248e+05,2.272320e+05,2.283106e+05,2.290993e+05,2.301417e+05,2.310600e+05,2.319275e+05,2.328221e+05,2.337123e+05,2.346912e+05,2.357603e+05,2.368364e+05,2.380098e+05,2.393542e+05,2.407441e+05,2.421035e+05,2.434244e+05,2.446583e+05,2.457646e+05,2.467650e+05,2.477652e+05,2.488964e+05,2.499873e+05,2.510143e+05,2.521643e+05,2.534836e+05,2.548201e+05,2.562058e+05,2.575979e+05,2.589460e+05,2.602543e+05,2.615769e+05,2.629238e+05,2.643733e+05,2.657551e+05,2.670651e+05,2.685335e+05,2.702268e+05,2.718247e+05,2.733354e+05,2.748658e+05,2.764646e+05,2.780332e+05,2.795209e+05,2.810953e+05,2.826571e+05,2.843687e+05,2.865114e+05,2.880399e+05
std,31934.118525,29359.325439,4250.308342,8.600251e+04,8.615567e+04,8.630923e+04,8.646795e+04,8.665094e+04,8.687208e+04,8.715185e+04,8.747981e+04,8.791269e+04,8.840935e+04,8.887373e+04,8.927704e+04,8.969700e+04,9.012583e+04,9.056147e+04,9.100389e+04,9.146850e+04,9.196320e+04,9.250608e+04,9.308367e+04,9.374830e+04,9.445247e+04,9.510499e+04,9.567485e+04,9.624843e+04,9.681400e+04,9.737949e+04,9.794880e+04,9.853754e+04,9.915791e+04,9.983302e+04,1.005517e+05,1.013752e+05,1.022670e+05,1.031269e+05,1.039398e+05,1.048064e+05,1.057199e+05,1.066999e+05,1.077590e+05,1.089064e+

In [7]:
class DataFrameChecker:
    def __init__(self, data):
        
        self.data = data

    def check_duplicates(self):
        
        duplicates = self.data.duplicated().any()
        return duplicates

    def check_missing_values(self):
        
        missing_values = self.data.isnull().values.any()
        return missing_values
    
checker = DataFrameChecker(df)

# Check for duplicates and missing values
duplicates = checker.check_duplicates()
missing_values = checker.check_missing_values()

if duplicates:
    print("Duplicates found in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

if missing_values:
    print("Missing values (NaNs) found in the DataFrame.")
else:
    print("No missing values (NaNs) found in the DataFrame.")

No duplicates found in the DataFrame.
Missing values (NaNs) found in the DataFrame.


In [9]:
df.isnull().sum()

RegionID         0
RegionName       0
City             0
State            0
Metro         1043
CountyName       0
SizeRank         0
1996-04       1039
1996-05       1039
1996-06       1039
1996-07       1039
1996-08       1039
1996-09       1039
1996-10       1039
1996-11       1039
1996-12       1039
1997-01       1039
1997-02       1039
1997-03       1039
1997-04       1039
1997-05       1039
1997-06       1039
1997-07       1038
1997-08       1038
1997-09       1038
1997-10       1038
1997-11       1038
1997-12       1038
1998-01       1036
1998-02       1036
1998-03       1036
1998-04       1036
1998-05       1036
1998-06       1036
1998-07       1036
1998-08       1036
1998-09       1036
1998-10       1036
1998-11       1036
1998-12       1036
1999-01       1036
1999-02       1036
1999-03       1036
1999-04       1036
1999-05       1036
1999-06       1036
1999-07       1036
1999-08       1036
1999-09       1036
1999-10       1036
1999-11       1036
1999-12       1036
2000-01     

## DATA PREPARATION

## 3.1 Data Cleaning

In [10]:
# Creating a Column of Actual ROI
df['ROIPrice'] = df["2018-04"] - df["2009-01"] - 1
df.ROIPrice

0         212299.0
1         119399.0
2          83199.0
3         241599.0
4           -101.0
5          40199.0
6          60499.0
7         108499.0
8          48499.0
9        1189299.0
10        407699.0
11         39899.0
12        427299.0
13        291899.0
14         67999.0
15        221199.0
16        122399.0
17        110099.0
18         70199.0
19         91799.0
20       5288699.0
21       1667499.0
22         71299.0
23         62499.0
24         43799.0
25         10699.0
26         16999.0
27        206199.0
28         -3301.0
29        322399.0
30        803999.0
31         62799.0
32         91999.0
33        386599.0
34         67599.0
35         74499.0
36             NaN
37         15699.0
38         64399.0
39         23799.0
40        268799.0
41         95999.0
42         37499.0
43         39499.0
44         63399.0
45         61499.0
46        207399.0
47        452599.0
48        -16101.0
49        297199.0
50         71599.0
51         52199.0
52        12

In [11]:
# Creating a Column of %ROI

df["%ROI"] = ((df["2018-04"] / df["2009-01"]) ** (1 / (2018-2009)) - 1) * 100
df["%ROI"]

0         2.596098
1         5.287075
2         3.281773
3         2.296917
4        -0.009141
5         3.166962
6         1.752813
7         1.671619
8         3.641813
9         4.240459
10        6.306338
11        1.935387
12        5.722477
13        5.040653
14        4.393741
15        5.065054
16        3.407095
17        6.173092
18        3.864598
19        2.477912
20        7.103007
21        2.883640
22        3.602271
23        3.151651
24        2.789320
25        0.652863
26        2.005069
27        7.523082
28       -0.282211
29        7.227562
30        6.013211
31        3.782252
32        4.199906
33        5.057521
34        4.657436
35        2.030348
36             NaN
37        1.434432
38        3.466856
39        1.482158
40        3.320466
41        3.427883
42        2.574221
43        3.154113
44        4.620447
45        3.148669
46        3.501133
47        5.429015
48       -1.036315
49        3.920764
50        1.892701
51        2.919140
52        4.

In [12]:
df.columns

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
       '2018-03', '2018-04', 'ROIPrice', '%ROI'],
      dtype='object', length=274)

In [13]:
# Rename the column using inplace=True
df.rename(columns={'RegionName': 'ZipCode'}, inplace=True)

In [14]:
# Convert Wide Format to Long format and checking on the first five rows
df_1 = pd.melt(df,
               id_vars=['RegionID','ZipCode', 'SizeRank', 'City', 'State', 'Metro', 'CountyName','%ROI','ROIPrice'],
               var_name='Date',
               value_name='Price'
               )

# convert date to datetime
df_1['Date'] = pd.to_datetime(df_1['Date'],format='%Y-%m')

# set date as index
#df_1 = df_1.set_index('Date')

# change Zipcode to string
df_1['ZipCode'] = df_1['ZipCode'].astype('str')

df_1.head()


,RegionID,ZipCode,SizeRank,City,State,Metro,CountyName,%ROI,ROIPrice,Date,Price
0,84654,60657,1,Chicago,IL,Chicago,Cook,2.596098,212299.0,1996-04-01,334200.0
1,90668,75070,2,McKinney,TX,Dallas-Fort Worth,Collin,5.287075,119399.0,1996-04-01,235700.0
2,91982,77494,3,Katy,TX,Houston,Harris,3.281773,83199.0,1996-04-01,210400.0
3,84616,60614,4,Chicago,IL,Chicago,Cook,2.296917,241599.0,1996-04-01,498100.0
4,93144,79936,5,El Paso,TX,El Paso,El Paso,-0.009141,-101.0,1996-04-01,77300.0


In [15]:
df_1.shape

(3901595, 11)

In [16]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3901595 entries, 0 to 3901594
Data columns (total 11 columns):
 #   Column      Dtype         
---  ------      -----         
 0   RegionID    int64         
 1   ZipCode     object        
 2   SizeRank    int64         
 3   City        object        
 4   State       object        
 5   Metro       object        
 6   CountyName  object        
 7   %ROI        float64       
 8   ROIPrice    float64       
 9   Date        datetime64[ns]
 10  Price       float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 327.4+ MB


In [17]:
df_1.describe()

,RegionID,SizeRank,%ROI,ROIPrice,Date,Price
count,3.901595e+06,3.901595e+06,3.746040e+06,3.746040e+06,3901595,3.744704e+06
mean,8.107501e+04,7.362000e+03,2.078260e+00,6.260119e+04,2007-04-01 18:12:13.584906496,2.076064e+05
min,5.819600e+04,1.000000e+00,-6.769992e+00,-5.328010e+05,1996-04-01 00:00:00,1.130000e+04
25%,6.717400e+04,3.681000e+03,6.961308e-01,9.199000e+03,2001-10-01 00:00:00,9.790000e+04
50%,7.800700e+04,7.362000e+03,1.989658e+00,2.829900e+04,2007-04-01 00:00:00,1.476000e+05
75%,9.092100e+04,1.104300e+04,3.306184e+00,6.579900e+04,2012-10-01 00:00:00,2.372000e+05
max,7.538440e+05,1.472300e+04,1.207663e+01,6.938599e+06,2018-04-01 00:00:00,1.931490e+07
std,3.193304e+04,4.250165e+03,2.060273e+00,1.624183e+05,NaN,2.400207e+05


In [18]:
#statistical description of categorical variables
df_1.describe(include=['object'])

,ZipCode,City,State,Metro,CountyName
count,3901595,3901595,3901595,3625200,3901595
unique,14723,7554,51,701,1212
top,60657,New York,CA,New York,Los Angeles
freq,265,30210,324360,206435,69960


In [20]:
checker = DataFrameChecker(df_1)

# Check for duplicates and missing values
duplicates = checker.check_duplicates()
missing_values = checker.check_missing_values()

if duplicates:
    print("Duplicates found in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

if missing_values:
    print("Missing values (NaNs) found in the DataFrame.")
else:
    print("No missing values (NaNs) found in the DataFrame.")

No duplicates found in the DataFrame.
Missing values (NaNs) found in the DataFrame.


In [21]:
# checking for percentage of missing values
df_1.isnull().mean() * 100

RegionID      0.000000
ZipCode       0.000000
SizeRank      0.000000
City          0.000000
State         0.000000
Metro         7.084154
CountyName    0.000000
%ROI          3.986959
ROIPrice      3.986959
Date          0.000000
Price         4.021202
dtype: float64

Metro has the highest percentage of missing values with a percentage of 7% while % ROI , ROI price and Price all have a percentage of 4 and below of missing values.

In [23]:
# Filling Missing Values in the metro column with "missing"
df_1.fillna({'Metro': 'Missing'},inplace = True)
df_1.isnull().mean()* 100

RegionID      0.000000
ZipCode       0.000000
SizeRank      0.000000
City          0.000000
State         0.000000
Metro         0.000000
CountyName    0.000000
%ROI          3.986959
ROIPrice      3.986959
Date          0.000000
Price         4.021202
dtype: float64

In [24]:
# Drop rows with null values
data = df_1.dropna()
data.isnull().mean()* 100

RegionID      0.0
ZipCode       0.0
SizeRank      0.0
City          0.0
State         0.0
Metro         0.0
CountyName    0.0
%ROI          0.0
ROIPrice      0.0
Date          0.0
Price         0.0
dtype: float64

Since rows with null values are less than 4% of the data we choose to drop them.